# Mean IOU for Semantic Segmetation Model
- [Brinkley97/PixelLib model GitHub Repo](https://github.com/Brinkley97/sem_seg_pxLib_ade20k/blob/main/ssForImg.ipynb)

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

## Load Semantic Segmetation Images
- Update method to load multiple images

In [2]:
def calc_iou(ground_truth_image, prediction_model_image):
    '''Determine the intersection over union when given a ground truth image and prediction model image
    
    Arguments:
    ground_truth_image -- string
    model_image -- string
    
    Return:
    iou
    
    '''
    
#    print(type(ground_truth_image))
#    print(type(prediction_model_image))
    
    '''
    Pre process ground_truth image for MeanIOU
    Get RGB to pass into TF MeanIOU function
    '''
    open_ground_truth_image = Image.open(ground_truth_image)
    ground_truth_rgb = np.asarray(open_ground_truth_image)
    # print("ground_truth_rgb size:", np.shape(ground_truth_rgb))
#    print("ground_truth_rgb image:", ground_truth_rgb)
    
    '''
    Pre process prediction_model image for MeanIOU
    Get RGB to pass into TF MeanIOU function
    '''
    open_prediction_model_image = Image.open(prediction_model_image)
    prediction_model_rgb = np.asarray(open_prediction_model_image)
    # print("prediction_model_rgb size:", np.shape(prediction_model_rgb))
#    print("prediction_model_rgb image:", prediction_model_rgb)
    
    if np.shape(ground_truth_rgb) != np.shape(prediction_model_rgb):
        # print(True)
        resized_prediction_model_rgb = np.resize(prediction_model_rgb, [550, 688, 3])
        # print("resized_prediction_model_rgb size:", np.shape(resized_prediction_model_rgb))
    else:
        resized_prediction_model_rgb = prediction_model_rgb
            

    
    '''
    Compute the mean intersection over union with TF
    https://www.tensorflow.org/api_docs/python/tf/keras/metrics/MeanIoU
    update_state method: https://github.com/keras-team/keras/blob/v2.10.0/keras/metrics/metrics.py#L2686-L2736
    '''
      
    mean_iou = tf.keras.metrics.MeanIoU(num_classes=550)
    mean_iou.update_state(ground_truth_rgb, resized_prediction_model_rgb)
    
    return mean_iou.result().numpy()

In [3]:
def iou(base, ground_truth_path, m_seg_path, ground_truth_directory, m_seg_directory):
    '''Show iou of each set of images
    
    Arguments:
    base -- string
    ground_truth_path -- string
    m_seg_path -- string
    ground_truth_images -- list
    prediction_model_image -- list
    
    Functions:
    calc_iou
    
    Return:
    iou for each set of images
    '''
    
    '''Set up path to get images
    
    '''
    # newline = print("\n")
    
    ground_truth_path += '{0}'
    m_seg_path += '{0}'
    
    ground_truth_images = [ground_truth_path.format(idx) for idx in ground_truth_directory]
    m_seg_images = [m_seg_path.format(idx) for idx in m_seg_directory]
    # print(ground_truth_images, '\n', m_seg_images)
    
    '''Store ground_truth_images as key and m_seg_images as value
    
    '''
    compare = {}
    store_iou = []
    
    for idx in range(len(ground_truth_images)):
        compare.update({ground_truth_images[idx]: m_seg_images[idx]})
    
    # print(compare.values())
    
    '''Pass images into calc_iou function
    
    '''
    for key, value in compare.items():
        ground_truth_image = key
        # print(ground_truth_image)
        
        prediction_model_image = value
        # print(prediction_model_image, "\n")
        
        i_o_u = calc_iou(ground_truth_image, prediction_model_image)
        store_iou.append(i_o_u)
    return compare, store_iou

In [4]:
base = "/Users/brinkley97/Documents/development/catalyst_robotics/iou/"
ground_truth_path = base + "ground_truth_rural_input_images/"
m_seg_path = base + "m_seg_rural_input_images/"

# directory of images
ground_truth_directory = os.listdir("ground_truth_rural_input_images/")
print(ground_truth_directory)

m_seg_directory = os.listdir("m_seg_rural_input_images/")
# print(m_seg_directory)

data, iou_with_m_seg = iou(base, ground_truth_path, m_seg_path, ground_truth_directory, m_seg_directory)
# iou_with_m_seg

['trail-6_00001.png', 'trail-14_01001.png', 'park-1_02001.png', 'trail-10_00001.png', 'trail-6_02001.png', 'park-1_00001.png', 'trail-11_02001.png', 'trail-7_00001.png', 'trail_02001.png', 'trail_00001.png', 'trail-11_00001.png', 'trail-4_00001.png', 'park-8_00001.png', 'trail-9_00001.png', 'park-2_00501.png', 'trail-4_02001.png', 'park-2_02001.png', 'creek_02001.png', 'trail-5_00001.png', 'trail-3_02001.png', 'trail-12_00501.png', 'trail-15_00001.png', 'trail-3_00001.png', 'village_00003.png', 'creek_00001.png', 'trail-13_00001.png']


2022-09-11 11:33:55.043923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def format_models_and_iou(ground_truth_directory, m_seg_directory, iou_with_m_seg):
    '''
    '''
    ground_truth = "Ground Truth and "
    m_seg_column = ground_truth + "M Seg"
    create_df = pd.DataFrame(iou_with_m_seg, columns=[m_seg_column], index=[ground_truth_directory])
    
    
    return create_df

In [6]:
format_models_and_iou(ground_truth_directory, m_seg_directory, iou_with_m_seg)

,Ground Truth and M Seg
trail-6_00001.png,0.000565
trail-14_01001.png,0.000579
park-1_02001.png,0.000565
trail-10_00001.png,0.000714
trail-6_02001.png,0.000612
park-1_00001.png,0.000533
trail-11_02001.png,0.000565
trail-7_00001.png,0.000573
trail_02001.png,0.000681
trail_00001.png,0.000675
